On prend le problème de prédiction du débit moyen journalier à une semaine, le critère que l'on va minimiser est un mean square error sur toutes les stations. (discuter)

On choisis de faire de la validation croisée pour faire de la sélection de modèle, et on garde un ensemble de test.  
On a des données de 2010 à 2020 (11 années).  

On garde 2020 comme ensemble de test, et on fait 10 ensembles de validation croisée, et on utilise le split time series de sklearn (prend le début et tester sur l'année suivante), ca fait une erreur calculée sur 9 splits.

Pour les évaluations, la première prédiction est celle du 15 Janvier, que l'on prédit à partir des données du 1 au 7 Janvier, on se place le 8 à 00h00. (on laisse une semaine de données pour calculer les features, et on prédit à J+7).

In [19]:
import pandas as pd
import numpy as np

In [20]:
stations_debit = pd.read_csv("../Data/Base/Stations_Debit.csv")
liste_stations = list(stations_debit["Code station"])
mesures = pd.read_csv("../Data/Base/Mesures.csv")
mesures["Date"] = pd.to_datetime(mesures["Date"], format = "%Y/%m/%d %H:%M:%S")

In [21]:
stations_debit

,Code station,Libelle station,Regime influencé,Altitude du zéro de l'échelle,geometry,Ordre,Cours eau
0,O0200020,La Garonne [partielle] à Saint-Gaudens [Valent...,pas ou faiblement,357.00,POINT (0.7068033357391884 43.09775183863328),0,Garonne
1,O2620010,La Garonne à Verdun-sur-Garonne,pas ou faiblement,90.00,POINT (1.242176195808334 43.85463266047086),1,Garonne
2,O6140010,La Garonne à Lamagistère,pas ou faiblement,46.00,POINT (0.8312973485763152 44.12107242091946),2,Garonne
3,O9000010,La Garonne à Tonneins,pas ou faiblement,0.00,POINT (0.2221525107266094 44.41192952559171),3,Garonne
4,K0030020,La Loire à Issarlès [Pont de La Borie],fortement,881.34,POINT (4.048486196061388 44.81973247281849),0,Loire
5,K0100020,La Loire à Goudet,fortement,0.00,POINT (3.921888161368209 44.88904936219829),1,Loire
6,K0260020,La Loire à Chadrac [Pont du Monteil à Chadrac],fortement en étiage,589.50,POINT (3.903694399697744 45.06795622583292),2,Loire
7,K0550010,La Loire à Bas-en-Basset,fortement,442.00,POINT (4.117990116352597 45.29522866665448),3,Loire
8,K0690010,La Loire à Montrond-les-Bains,fortement,339.11,POINT (4.225355804253323 45.64333335504166),4,Loire
9,K0910050,La Loire à Villerest [Pont de Villerest],fortement,267.51,POINT (4.046217438918319 45.99586067270341),5,Loire


In [22]:
stations_meteo = pd.read_csv("../Data/Base/Stations_Meteo.csv")
colonnes_meteo = ["Pression", "Vent_Nord", "Vent_Est", "Vitesse_vent", "Temperature", "Humidite", "Precipitations"]
liste_stations_meteo = list(stations_meteo["ID"].apply(lambda x: str(x).rjust(5, "0")))
stations_meteo

,ID,Nom,Latitude,Longitude,Altitude,geometry
0,7005,ABBEVILLE,50.136000,1.834000,69,POINT (1.834 50.136)
1,7015,LILLE-LESQUIN,50.570000,3.097500,47,POINT (3.0975 50.57)
2,7027,CAEN-CARPIQUET,49.180000,-0.456167,67,POINT (-0.456167 49.18)
3,7037,ROUEN-BOOS,49.383000,1.181667,151,POINT (1.181667 49.383)
4,7110,BREST-GUIPAVAS,48.444167,-4.412000,94,POINT (-4.412 48.444167)
5,7117,PLOUMANAC'H,48.825833,-3.473167,55,POINT (-3.473167 48.825833)
6,7130,RENNES-ST JACQUES,48.068833,-1.734000,36,POINT (-1.734 48.068833)
7,7139,ALENCON,48.445500,0.110167,143,POINT (0.110167 48.4455)
8,7149,ORLY,48.716833,2.384333,89,POINT (2.384333 48.716833)
9,7168,TROYES-BARBEREY,48.324667,4.020000,112,POINT (4.02 48.324667)


## Ensemble de test

In [23]:
mesures_test = mesures[mesures["Date"].apply(lambda x: x.year == 2020)]

In [24]:
mesures_test_X = mesures_test.copy()
mesures_test_X

,Date,O0200020,O2620010,O6140010,O9000010,K0030020,K0100020,K0260020,K0550010,K0690010,...,Temperature_07747,Humidite_07747,Precipitations_07747,Pression_07761,Vent_Nord_07761,Vent_Est_07761,Vitesse_vent_07761,Temperature_07761,Humidite_07761,Precipitations_07761
3652,2020-01-01,57.383333,191.166667,591.333333,1068.333333,1.440833,6.097500,18.541667,45.708333,87.825000,...,7.2875,77.875,0.0,103048.75,1.486515,1.180520,2.8750,8.3375,81.125,0.0000
3653,2020-01-02,55.450000,172.083333,516.250000,839.333333,1.362500,5.724167,17.175000,44.608333,86.675000,...,7.0375,85.000,0.0,103136.25,1.555799,1.256975,2.8750,8.2875,81.375,0.0000
3654,2020-01-03,51.900000,168.416667,491.750000,841.666667,1.305000,5.469167,16.350000,44.308333,75.691667,...,10.0875,70.500,0.0,102917.50,1.581900,1.310879,2.4375,9.4875,82.375,0.0000
3655,2020-01-04,48.725000,164.750000,461.750000,726.500000,1.272500,5.360000,16.075000,40.075000,61.933333,...,12.6625,61.125,0.0,102375.00,0.972261,0.740312,2.2125,11.6250,78.000,0.0000
3656,2020-01-05,47.183333,148.750000,442.833333,681.500000,1.220833,4.981667,15.000000,36.591667,67.475000,...,11.3000,49.000,0.0,102521.25,1.334908,0.695272,2.7000,9.8750,83.125,0.0000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4013,2020-12-27,41.291667,188.250000,378.833333,627.666667,0.672500,3.170000,13.083333,30.383333,69.258333,...,4.0625,58.375,0.0,100906.25,1.687684,0.985724,2.8000,5.9500,69.750,-0.0125
4014,2020-12-28,51.316667,243.500000,425.666667,676.250000,0.683333,3.257500,13.516667,32.983333,85.591667,...,6.7000,60.875,0.0,99291.25,-3.083568,-4.526198,7.4750,11.5125,77.750,1.0375
4015,2020-12-29,47.250000,552.666667,1474.750000,1975.833333,0.702500,3.126667,13.058333,35.491667,83.050000,...,8.4625,54.625,0.0,99955.00,2.336237,1.208835,2.7000,8.4250,89.250,0.3125
4016,2020-12-30,47.225000,359.583333,1192.916667,2684.166667,0.760833,3.132500,12.375000,32.058333,43.616667,...,7.6375,57.375,0.0,100448.75,1.686884,1.855508,2.7750,7.1750,87.375,0.3125


In [25]:
mesures_test_X.to_csv("../Data/Base/Mesures_Test_X.csv",
                     index = False)

In [26]:
mesures_test_Y = mesures_test[(mesures_test["Date"].apply(lambda x: x.month != 1)) \
                                | (mesures_test["Date"].apply(lambda x: x.day >= 15))]
mesures_test_Y

,Date,O0200020,O2620010,O6140010,O9000010,K0030020,K0100020,K0260020,K0550010,K0690010,...,Temperature_07747,Humidite_07747,Precipitations_07747,Pression_07761,Vent_Nord_07761,Vent_Est_07761,Vitesse_vent_07761,Temperature_07761,Humidite_07761,Precipitations_07761
3666,2020-01-15,32.558333,125.416667,303.500000,518.833333,0.962500,3.465000,9.922500,30.966667,33.968333,...,6.5750,78.125,0.000,102775.00,1.243892,0.617187,3.2125,8.4125,75.250,0.0000
3667,2020-01-16,34.158333,116.583333,322.416667,507.250000,0.943333,3.218333,9.325000,29.525000,34.380000,...,7.7000,88.750,0.000,102920.00,1.959215,1.468298,2.4625,9.9750,76.875,-0.0125
3668,2020-01-17,36.283333,128.500000,319.000000,546.166667,0.958333,3.280000,9.489167,27.950000,35.800000,...,9.4375,77.000,0.025,102647.50,0.066474,0.672485,2.6750,11.7250,81.500,0.0625
3669,2020-01-18,38.525000,138.750000,345.583333,534.583333,0.927500,3.451667,11.091667,28.375000,70.450000,...,9.0250,62.500,0.000,102312.50,1.218467,-2.505142,3.8125,11.1000,68.875,0.0500
3670,2020-01-19,34.666667,138.333333,335.083333,575.500000,0.888333,3.070000,9.660000,26.816667,68.816667,...,8.5250,46.625,0.000,102135.00,1.283513,1.229322,2.9000,7.2500,70.500,0.0000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4013,2020-12-27,41.291667,188.250000,378.833333,627.666667,0.672500,3.170000,13.083333,30.383333,69.258333,...,4.0625,58.375,0.000,100906.25,1.687684,0.985724,2.8000,5.9500,69.750,-0.0125
4014,2020-12-28,51.316667,243.500000,425.666667,676.250000,0.683333,3.257500,13.516667,32.983333,85.591667,...,6.7000,60.875,0.000,99291.25,-3.083568,-4.526198,7.4750,11.5125,77.750,1.0375
4015,2020-12-29,47.250000,552.666667,1474.750000,1975.833333,0.702500,3.126667,13.058333,35.491667,83.050000,...,8.4625,54.625,0.000,99955.00,2.336237,1.208835,2.7000,8.4250,89.250,0.3125
4016,2020-12-30,47.225000,359.583333,1192.916667,2684.166667,0.760833,3.132500,12.375000,32.058333,43.616667,...,7.6375,57.375,0.000,100448.75,1.686884,1.855508,2.7750,7.1750,87.375,0.3125


In [27]:
mesures_test_Y.to_csv("../Data/Base/Mesures_Test_Y.csv",
                     index = False)

## Ensemble d'entrainement

In [28]:
mesures_train = mesures[mesures["Date"].apply(lambda x: x.year < 2020)]
mesures_train

,Date,O0200020,O2620010,O6140010,O9000010,K0030020,K0100020,K0260020,K0550010,K0690010,...,Temperature_07747,Humidite_07747,Precipitations_07747,Pression_07761,Vent_Nord_07761,Vent_Est_07761,Vitesse_vent_07761,Temperature_07761,Humidite_07761,Precipitations_07761
0,2010-01-01,42.375000,128.000000,350.500000,490.583333,0.535000,3.119167,11.005000,24.850000,8.470833,...,10.8375,54.500,0.025,99478.75,-2.842313,-10.281279,11.5000,12.4250,78.125,0.550
1,2010-01-02,38.458333,106.175000,447.666667,666.333333,0.467500,2.952500,10.812500,29.208333,8.045000,...,5.2250,62.500,0.000,100988.75,0.439673,-5.555278,6.0375,11.0000,56.875,0.000
2,2010-01-03,35.891667,104.941667,385.000000,690.333333,0.423333,2.385833,8.619167,21.000000,7.295833,...,4.9250,66.875,0.000,101792.50,1.802059,1.524495,3.3375,6.5750,64.500,0.000
3,2010-01-04,34.041667,92.708333,343.333333,607.833333,0.424167,2.447500,8.489167,21.616667,42.940000,...,5.4000,83.750,0.025,101070.00,1.809303,1.632121,2.4625,8.9750,85.625,0.075
4,2010-01-05,37.175000,109.258333,357.083333,616.666667,0.421667,2.426667,8.027500,23.250000,56.650000,...,6.7750,86.875,0.325,99836.25,0.903730,1.437407,2.1750,10.8500,93.125,1.150
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3647,2019-12-27,81.958333,292.583333,955.000000,1720.833333,1.997500,9.271667,28.816667,60.675000,102.166667,...,12.7500,58.875,0.000,102098.75,1.497415,0.985754,2.5750,9.4625,83.625,0.025
3648,2019-12-28,75.508333,258.250000,849.583333,1530.833333,1.912500,8.575000,27.591667,61.533333,102.983333,...,11.1125,72.875,0.000,102655.00,1.637983,1.242350,2.5750,9.5000,81.250,0.000
3649,2019-12-29,69.008333,236.333333,749.416667,1375.000000,1.784167,7.627500,24.133333,53.983333,93.641667,...,7.7000,82.750,0.000,103027.50,1.317747,1.181957,2.7375,9.5125,73.875,0.000
3650,2019-12-30,63.858333,215.000000,682.000000,1245.000000,1.650833,6.958333,21.900000,49.550000,90.725000,...,6.8750,85.375,0.000,103025.00,1.695063,1.365617,2.9875,6.3250,74.125,0.000


In [29]:
mesures_train_X = mesures_train.copy()
mesures_train_X.to_csv("../Data/Base/Mesures_Train_X.csv",
                     index = False)
mesures_train_X

,Date,O0200020,O2620010,O6140010,O9000010,K0030020,K0100020,K0260020,K0550010,K0690010,...,Temperature_07747,Humidite_07747,Precipitations_07747,Pression_07761,Vent_Nord_07761,Vent_Est_07761,Vitesse_vent_07761,Temperature_07761,Humidite_07761,Precipitations_07761
0,2010-01-01,42.375000,128.000000,350.500000,490.583333,0.535000,3.119167,11.005000,24.850000,8.470833,...,10.8375,54.500,0.025,99478.75,-2.842313,-10.281279,11.5000,12.4250,78.125,0.550
1,2010-01-02,38.458333,106.175000,447.666667,666.333333,0.467500,2.952500,10.812500,29.208333,8.045000,...,5.2250,62.500,0.000,100988.75,0.439673,-5.555278,6.0375,11.0000,56.875,0.000
2,2010-01-03,35.891667,104.941667,385.000000,690.333333,0.423333,2.385833,8.619167,21.000000,7.295833,...,4.9250,66.875,0.000,101792.50,1.802059,1.524495,3.3375,6.5750,64.500,0.000
3,2010-01-04,34.041667,92.708333,343.333333,607.833333,0.424167,2.447500,8.489167,21.616667,42.940000,...,5.4000,83.750,0.025,101070.00,1.809303,1.632121,2.4625,8.9750,85.625,0.075
4,2010-01-05,37.175000,109.258333,357.083333,616.666667,0.421667,2.426667,8.027500,23.250000,56.650000,...,6.7750,86.875,0.325,99836.25,0.903730,1.437407,2.1750,10.8500,93.125,1.150
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3647,2019-12-27,81.958333,292.583333,955.000000,1720.833333,1.997500,9.271667,28.816667,60.675000,102.166667,...,12.7500,58.875,0.000,102098.75,1.497415,0.985754,2.5750,9.4625,83.625,0.025
3648,2019-12-28,75.508333,258.250000,849.583333,1530.833333,1.912500,8.575000,27.591667,61.533333,102.983333,...,11.1125,72.875,0.000,102655.00,1.637983,1.242350,2.5750,9.5000,81.250,0.000
3649,2019-12-29,69.008333,236.333333,749.416667,1375.000000,1.784167,7.627500,24.133333,53.983333,93.641667,...,7.7000,82.750,0.000,103027.50,1.317747,1.181957,2.7375,9.5125,73.875,0.000
3650,2019-12-30,63.858333,215.000000,682.000000,1245.000000,1.650833,6.958333,21.900000,49.550000,90.725000,...,6.8750,85.375,0.000,103025.00,1.695063,1.365617,2.9875,6.3250,74.125,0.000


In [30]:
mesures_train_Y = mesures_train.copy()
mesures_train_Y.to_csv("../Data/Base/Mesures_Train_Y.csv",
                     index = False)

mesures_train_Y

,Date,O0200020,O2620010,O6140010,O9000010,K0030020,K0100020,K0260020,K0550010,K0690010,...,Temperature_07747,Humidite_07747,Precipitations_07747,Pression_07761,Vent_Nord_07761,Vent_Est_07761,Vitesse_vent_07761,Temperature_07761,Humidite_07761,Precipitations_07761
0,2010-01-01,42.375000,128.000000,350.500000,490.583333,0.535000,3.119167,11.005000,24.850000,8.470833,...,10.8375,54.500,0.025,99478.75,-2.842313,-10.281279,11.5000,12.4250,78.125,0.550
1,2010-01-02,38.458333,106.175000,447.666667,666.333333,0.467500,2.952500,10.812500,29.208333,8.045000,...,5.2250,62.500,0.000,100988.75,0.439673,-5.555278,6.0375,11.0000,56.875,0.000
2,2010-01-03,35.891667,104.941667,385.000000,690.333333,0.423333,2.385833,8.619167,21.000000,7.295833,...,4.9250,66.875,0.000,101792.50,1.802059,1.524495,3.3375,6.5750,64.500,0.000
3,2010-01-04,34.041667,92.708333,343.333333,607.833333,0.424167,2.447500,8.489167,21.616667,42.940000,...,5.4000,83.750,0.025,101070.00,1.809303,1.632121,2.4625,8.9750,85.625,0.075
4,2010-01-05,37.175000,109.258333,357.083333,616.666667,0.421667,2.426667,8.027500,23.250000,56.650000,...,6.7750,86.875,0.325,99836.25,0.903730,1.437407,2.1750,10.8500,93.125,1.150
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3647,2019-12-27,81.958333,292.583333,955.000000,1720.833333,1.997500,9.271667,28.816667,60.675000,102.166667,...,12.7500,58.875,0.000,102098.75,1.497415,0.985754,2.5750,9.4625,83.625,0.025
3648,2019-12-28,75.508333,258.250000,849.583333,1530.833333,1.912500,8.575000,27.591667,61.533333,102.983333,...,11.1125,72.875,0.000,102655.00,1.637983,1.242350,2.5750,9.5000,81.250,0.000
3649,2019-12-29,69.008333,236.333333,749.416667,1375.000000,1.784167,7.627500,24.133333,53.983333,93.641667,...,7.7000,82.750,0.000,103027.50,1.317747,1.181957,2.7375,9.5125,73.875,0.000
3650,2019-12-30,63.858333,215.000000,682.000000,1245.000000,1.650833,6.958333,21.900000,49.550000,90.725000,...,6.8750,85.375,0.000,103025.00,1.695063,1.365617,2.9875,6.3250,74.125,0.000


## Ensembles de validation croisée

In [31]:
mesures_train_X = pd.read_csv("../Data/Base/Mesures_Train_X.csv")
mesures_train_X["Date"] = pd.to_datetime(mesures_train_X["Date"], format = "%Y/%m/%d %H:%M:%S")
mesures_train_X

,Date,O0200020,O2620010,O6140010,O9000010,K0030020,K0100020,K0260020,K0550010,K0690010,...,Temperature_07747,Humidite_07747,Precipitations_07747,Pression_07761,Vent_Nord_07761,Vent_Est_07761,Vitesse_vent_07761,Temperature_07761,Humidite_07761,Precipitations_07761
0,2010-01-01,42.375000,128.000000,350.500000,490.583333,0.535000,3.119167,11.005000,24.850000,8.470833,...,10.8375,54.500,0.025,99478.75,-2.842313,-10.281279,11.5000,12.4250,78.125,0.550
1,2010-01-02,38.458333,106.175000,447.666667,666.333333,0.467500,2.952500,10.812500,29.208333,8.045000,...,5.2250,62.500,0.000,100988.75,0.439673,-5.555278,6.0375,11.0000,56.875,0.000
2,2010-01-03,35.891667,104.941667,385.000000,690.333333,0.423333,2.385833,8.619167,21.000000,7.295833,...,4.9250,66.875,0.000,101792.50,1.802059,1.524495,3.3375,6.5750,64.500,0.000
3,2010-01-04,34.041667,92.708333,343.333333,607.833333,0.424167,2.447500,8.489167,21.616667,42.940000,...,5.4000,83.750,0.025,101070.00,1.809303,1.632121,2.4625,8.9750,85.625,0.075
4,2010-01-05,37.175000,109.258333,357.083333,616.666667,0.421667,2.426667,8.027500,23.250000,56.650000,...,6.7750,86.875,0.325,99836.25,0.903730,1.437407,2.1750,10.8500,93.125,1.150
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3647,2019-12-27,81.958333,292.583333,955.000000,1720.833333,1.997500,9.271667,28.816667,60.675000,102.166667,...,12.7500,58.875,0.000,102098.75,1.497415,0.985754,2.5750,9.4625,83.625,0.025
3648,2019-12-28,75.508333,258.250000,849.583333,1530.833333,1.912500,8.575000,27.591667,61.533333,102.983333,...,11.1125,72.875,0.000,102655.00,1.637983,1.242350,2.5750,9.5000,81.250,0.000
3649,2019-12-29,69.008333,236.333333,749.416667,1375.000000,1.784167,7.627500,24.133333,53.983333,93.641667,...,7.7000,82.750,0.000,103027.50,1.317747,1.181957,2.7375,9.5125,73.875,0.000
3650,2019-12-30,63.858333,215.000000,682.000000,1245.000000,1.650833,6.958333,21.900000,49.550000,90.725000,...,6.8750,85.375,0.000,103025.00,1.695063,1.365617,2.9875,6.3250,74.125,0.000


In [32]:
train_split_X = pd.DataFrame()
for i in range(9):
    train_split_X["Train_" + str(i)] = mesures_train_X["Date"].apply(lambda x: x.year <= 2010+i)
for i in range(9):
    train_split_X["Test_" + str(i)] = mesures_train_X["Date"].apply(lambda x: x.year == 2010+i+1)
train_split_X

,Train_0,Train_1,Train_2,Train_3,Train_4,Train_5,Train_6,Train_7,Train_8,Test_0,Test_1,Test_2,Test_3,Test_4,Test_5,Test_6,Test_7,Test_8
0,True,True,True,True,True,True,True,True,True,False,False,False,False,False,False,False,False,False
1,True,True,True,True,True,True,True,True,True,False,False,False,False,False,False,False,False,False
2,True,True,True,True,True,True,True,True,True,False,False,False,False,False,False,False,False,False
3,True,True,True,True,True,True,True,True,True,False,False,False,False,False,False,False,False,False
4,True,True,True,True,True,True,True,True,True,False,False,False,False,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3647,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True
3648,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True
3649,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True
3650,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True


In [33]:
train_split_X.to_csv("../Data/Base/Index_CV_X.csv",
                    index=False)

In [34]:
mesures_train_Y = pd.read_csv("../Data/Base/Mesures_Train_Y.csv")
mesures_train_Y["Date"] = pd.to_datetime(mesures_train_Y["Date"], format = "%Y/%m/%d %H:%M:%S")
mesures_train_Y

,Date,O0200020,O2620010,O6140010,O9000010,K0030020,K0100020,K0260020,K0550010,K0690010,...,Temperature_07747,Humidite_07747,Precipitations_07747,Pression_07761,Vent_Nord_07761,Vent_Est_07761,Vitesse_vent_07761,Temperature_07761,Humidite_07761,Precipitations_07761
0,2010-01-01,42.375000,128.000000,350.500000,490.583333,0.535000,3.119167,11.005000,24.850000,8.470833,...,10.8375,54.500,0.025,99478.75,-2.842313,-10.281279,11.5000,12.4250,78.125,0.550
1,2010-01-02,38.458333,106.175000,447.666667,666.333333,0.467500,2.952500,10.812500,29.208333,8.045000,...,5.2250,62.500,0.000,100988.75,0.439673,-5.555278,6.0375,11.0000,56.875,0.000
2,2010-01-03,35.891667,104.941667,385.000000,690.333333,0.423333,2.385833,8.619167,21.000000,7.295833,...,4.9250,66.875,0.000,101792.50,1.802059,1.524495,3.3375,6.5750,64.500,0.000
3,2010-01-04,34.041667,92.708333,343.333333,607.833333,0.424167,2.447500,8.489167,21.616667,42.940000,...,5.4000,83.750,0.025,101070.00,1.809303,1.632121,2.4625,8.9750,85.625,0.075
4,2010-01-05,37.175000,109.258333,357.083333,616.666667,0.421667,2.426667,8.027500,23.250000,56.650000,...,6.7750,86.875,0.325,99836.25,0.903730,1.437407,2.1750,10.8500,93.125,1.150
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3647,2019-12-27,81.958333,292.583333,955.000000,1720.833333,1.997500,9.271667,28.816667,60.675000,102.166667,...,12.7500,58.875,0.000,102098.75,1.497415,0.985754,2.5750,9.4625,83.625,0.025
3648,2019-12-28,75.508333,258.250000,849.583333,1530.833333,1.912500,8.575000,27.591667,61.533333,102.983333,...,11.1125,72.875,0.000,102655.00,1.637983,1.242350,2.5750,9.5000,81.250,0.000
3649,2019-12-29,69.008333,236.333333,749.416667,1375.000000,1.784167,7.627500,24.133333,53.983333,93.641667,...,7.7000,82.750,0.000,103027.50,1.317747,1.181957,2.7375,9.5125,73.875,0.000
3650,2019-12-30,63.858333,215.000000,682.000000,1245.000000,1.650833,6.958333,21.900000,49.550000,90.725000,...,6.8750,85.375,0.000,103025.00,1.695063,1.365617,2.9875,6.3250,74.125,0.000


In [35]:
train_split_Y = pd.DataFrame()
for i in range(9):
    train_split_Y["Train_" + str(i)] = mesures_train_Y["Date"].apply(lambda x: x.year <= 2010+i)
    train_split_Y["Train_" + str(i)].iloc[mesures_train_Y["Date"].apply(lambda x: x.month == 1) & mesures_train_Y["Date"].apply(lambda x: x.day < 15)] = False # On enlève les 14 premiers jours
    
for i in range(9):
    train_split_Y["Test_" + str(i)] = mesures_train_Y["Date"].apply(lambda x: x.year == 2010+i+1)
    train_split_Y["Test_" + str(i)].iloc[mesures_train_Y["Date"].apply(lambda x: x.month == 1) & mesures_train_Y["Date"].apply(lambda x: x.day < 15)] = False  # On enlève les 14 premiers jours
train_split_Y

,Train_0,Train_1,Train_2,Train_3,Train_4,Train_5,Train_6,Train_7,Train_8,Test_0,Test_1,Test_2,Test_3,Test_4,Test_5,Test_6,Test_7,Test_8
0,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
1,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
2,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
3,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
4,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3647,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True
3648,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True
3649,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True
3650,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True


In [36]:
train_split_Y.to_csv("../Data/Base/Index_CV_Y.csv",
                    index=False)

# Métriques

- RMSE
- MAE
- R^2